# Импорт библиотек

In [19]:
import re
import shap
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from ydata_profiling import ProfileReport
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import FeatureDrift
from imblearn.over_sampling import SMOTENC, RandomOverSampler

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, 
    accuracy_score, 
    confusion_matrix, 
    f1_score,
    precision_recall_curve,
    roc_curve, 
    auc
)
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from catboost import (
    CatBoostClassifier,
    Pool, 
    cv, 
    EShapCalcType, 
    EFeaturesSelectionAlgorithm
)


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 200)

# Чтение данных

In [20]:
train_df = pd.read_csv('data/train.csv', sep=";", decimal=",", encoding="windows-1251")
test_df = pd.read_csv('data/test.csv', sep=";", decimal=",", encoding="windows-1251")

features_description = (
    pd.read_excel('data/description.xlsx', index_col='field')
    .to_dict()['description']
)

train_df.shape, test_df.shape

((205962, 235), (37183, 233))

# Предварительный анализ EDA

In [21]:
features = list(train_df.drop(['client_id', 'feature_date', 'target', 'w'], axis=1).columns)

### Поправляем типы данных для фичей

In [22]:
newtype_feats = [
    'bki_total_ip_max_limit',
    'hdb_bki_active_cc_cnt',
    'hdb_bki_active_ip_max_outstand',
    'hdb_bki_active_micro_max_outstand',
    'hdb_bki_active_pil_max_overdue',
    'hdb_bki_other_active_auto_month_payments_sum',
    'hdb_bki_total_cc_max_limit',
    'hdb_bki_total_ip_cnt',
    'hdb_bki_total_max_limit',
    'hdb_bki_total_max_overdue_sum',
    'hdb_bki_total_pil_max_limit'
]

test_df[newtype_feats] = test_df[newtype_feats].astype('float64')
test_df['feature_date'] = test_df['feature_date'].astype('datetime64[ns]')

train_df[newtype_feats] = train_df[newtype_feats].astype('float64')
train_df['feature_date'] = train_df['feature_date'].astype('datetime64[ns]')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205962 entries, 0 to 205961
Columns: 235 entries, client_id to productionyear
dtypes: datetime64[ns](1), float64(223), int64(1), object(10)
memory usage: 369.3+ MB


In [23]:
cat_features = list(train_df[features].select_dtypes('object').columns)
num_features = list(set(features) - set(cat_features))

len(features), len(cat_features), len(num_features)

(231, 10, 221)

### Анализ категорильных фичей

In [24]:
# сколько всего записей, где известна должность, но не известна зп?
train_df.loc[
    (train_df['worksalary_rur_amt'].isna())
    & (~train_df['part_last_position_ccode'].isna())
].shape

(4184, 235)

In [25]:
# df, содержащий медианные зарплаты для должностей по всем данным
median_worksalary = (
    pd.concat([train_df[features], test_df[features]], axis=0)
    .dropna(subset = 'worksalary_rur_amt')
    .groupby('part_last_position_ccode', as_index=False)
    ['worksalary_rur_amt'].median()
    .rename(columns={'worksalary_rur_amt': 'median_worksalary_rur_amt'})
)
median_worksalary.head()

,part_last_position_ccode,median_worksalary_rur_amt
0,АВТОМЕХАНИК,158892.5
1,АДМИНИСТРАТОР,180000.0
2,АНАЛИТИК,100000.0
3,Администратор,60000.0
4,Администратор магазина,70000.0


In [26]:
test_df = test_df.merge(median_worksalary, on='part_last_position_ccode', how='left')
test_df['worksalary_rur_amt'] = test_df['worksalary_rur_amt'].fillna(test_df['median_worksalary_rur_amt'])
test_df.drop('median_worksalary_rur_amt', axis=1, inplace=True)

train_df = train_df.merge(median_worksalary, on='part_last_position_ccode', how='left')
train_df['worksalary_rur_amt'] = train_df['worksalary_rur_amt'].fillna(train_df['median_worksalary_rur_amt'])
train_df.drop('median_worksalary_rur_amt', axis=1, inplace=True)

train_df.loc[
    (train_df['worksalary_rur_amt'].isna())
    & (~train_df['part_last_position_ccode'].isna())
].shape

(1029, 235)

In [27]:
# заполняем пропуски в категориальных
train_df['addrref'] = train_df['addrref'].fillna('Россия')
test_df['addrref'] = test_df['addrref'].fillna('Россия')

train_df[cat_features] = train_df[cat_features].fillna('miss_value')
test_df[cat_features] = test_df[cat_features].fillna('miss_value')

# Инженерия фич

### Фичи по датасету

In [28]:
# извлекаем фичи из дат
train_df['feature_date_year'] = train_df['feature_date'].dt.year
train_df['feature_date_month'] = train_df['feature_date'].dt.month
train_df['feature_date_day'] = train_df['feature_date'].dt.day

test_df['feature_date_year'] = test_df['feature_date'].dt.year
test_df['feature_date_month'] = test_df['feature_date'].dt.month
test_df['feature_date_day'] = test_df['feature_date'].dt.day

# машина
train_df['car_old'] = train_df['feature_date_year'] - train_df['productionyear']
train_df['car_country'] = train_df['brand']
train_df.loc[
    (train_df['car_country'].str.contains('LADA', flags=re.IGNORECASE))
    | (train_df['car_country'].str.contains('ЛАДА', flags=re.IGNORECASE))
    | (train_df['car_country'].isin(['МОСКВИЧ','КАМАЗ','ЗАЗ', 'ГАЗ'])),
    'car_country'
] = 'Россия'
train_df.loc[
    ~train_df['car_country'].isin(['miss_value', 'Россия']),
    'car_country'
] = 'Иномарка'

test_df['car_old'] = test_df['feature_date_year'] - test_df['productionyear']
test_df['car_country'] = test_df['brand']
test_df.loc[
    (test_df['car_country'].str.contains('LADA', flags=re.IGNORECASE))
    | (test_df['car_country'].str.contains('ЛАДА', flags=re.IGNORECASE))
    | (test_df['car_country'].isin(['МОСКВИЧ','КАМАЗ','ЗАЗ', 'ГАЗ'])),
    'car_country'
] = 'Россия'
test_df.loc[
    ~test_df['car_country'].isin(['miss_value', 'Россия']),
    'car_country'
] = 'Иномарка'

# средняя, медианная, минимальная, максимальная зарплата по должности
train_df = (
    train_df
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'position_median_worksalary_rur_amt'}),
        on='part_last_position_ccode', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'position_mean_worksalary_rur_amt'}),
        on='part_last_position_ccode', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'position_max_worksalary_rur_amt'}),
        on='part_last_position_ccode', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'position_min_worksalary_rur_amt'}),
        on='part_last_position_ccode', 
        how='left'
    )
)

test_df = (
    test_df
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'position_median_worksalary_rur_amt'}),
        on='part_last_position_ccode', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'position_mean_worksalary_rur_amt'}),
        on='part_last_position_ccode', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'position_max_worksalary_rur_amt'}),
        on='part_last_position_ccode', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'position_min_worksalary_rur_amt'}),
        on='part_last_position_ccode', 
        how='left'
    )
)

# средняя, медианная, минимальная, максимальная зарплата по месту жительства
train_df = (
    train_df
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'addrref_median_worksalary_rur_amt'}),
        on='addrref', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'addrref_mean_worksalary_rur_amt'}),
        on='addrref', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'addrref_max_worksalary_rur_amt'}),
        on='addrref', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'addrref_min_worksalary_rur_amt'}),
        on='addrref', 
        how='left'
    )
)

test_df = (
    test_df
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'addrref_median_worksalary_rur_amt'}),
        on='addrref', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'addrref_mean_worksalary_rur_amt'}),
        on='addrref', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'addrref_max_worksalary_rur_amt'}),
        on='addrref', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'addrref_min_worksalary_rur_amt'}),
        on='addrref', 
        how='left'
    )
)

# средняя, медианная, минимальная, максимальная зарплата по сегменту
train_df = (
    train_df
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'segment_median_worksalary_rur_amt'}),
        on='segment', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'segment_mean_worksalary_rur_amt'}),
        on='segment', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'segment_max_worksalary_rur_amt'}),
        on='segment', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'segment_min_worksalary_rur_amt'}),
        on='segment', 
        how='left'
    )
)

test_df = (
    test_df
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'segment_median_worksalary_rur_amt'}),
        on='segment', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'segment_mean_worksalary_rur_amt'}),
        on='segment', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'segment_max_worksalary_rur_amt'}),
        on='segment', 
        how='left'
    )
    .merge(
        pd.concat([train_df[features], test_df[features]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'segment_min_worksalary_rur_amt'}),
        on='segment', 
        how='left'
    )
)

In [30]:
# Пробуем создать новую фичу с среднемесячной суммой трат по известным категорям + выдача наличных в банкомате + среднемесячные платежи по кредитам(автокредиты)
# Т.е. мы  объединяем все маловажные по значимости(около 0) фичи в одну общую, которая по итогам обучения получала от 7 до 15 места в списке значимости

# Для создания этой фичи нам потребуются заполненные 0 значения Nan. Создадим копии исходных датафремов и заполним пропуски 0
train_df2 = train_df[num_features].copy()
test_df2 = test_df[num_features].copy()
train_df2 = train_df2[num_features].fillna(0).astype(float)
test_df2 = test_df2[num_features].fillna(0).astype(float)


# Определяем колонки для суммирования по всем известным среднемесячным тратам
columns_to_sum = [
    'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__bilety_na_kontserty_i_v_teatry',
    'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__brokerskie_uslugi',
    'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__hosting',
    'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__spa_sauny_bani',
    'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__tovary_dlja_detej',
    'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__turisticheskie_agenstva',
    'avg_3m_hotels',
    'avg_6m_building_services',
    'avg_6m_money_transactions',
    'avg_6m_personal_services',
    'avg_6m_transportation',
    'avg_by_category__amount__sum__cashflowcategory_name__detskie_igrushki',
    'avg_by_category__amount__sum__cashflowcategory_name__investitsii',
    'avg_by_category__amount__sum__cashflowcategory_name__odezhda_dlja_beremennyh',
    'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate',
    'avg_by_category__amount__sum__cashflowcategory_name__zdorove',
    'avg_by_category__amount__sum__cashflowcategory_name__zooparki',
    'by_category__amount__sum__eoperation_type_name__platezh_cherez_vidzhet_moj_mobilnyj',
    'by_category__amount__sum__eoperation_type_name__pokupka_paja',
    'by_category__amount__sum__eoperation_type_name__vneshnij_perevod_rur',
    'summarur_1m_miscellaneous_stores',
    'summarur_1m_no_cat',
    'hdb_bki_other_active_auto_month_payments_sum' # Средний платёж по автокредиту в других банках
]
feats_90_to_30 =[
    'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__marketplejsy',
    'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__nalogi',
    'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__ohota_i_rybalka',
    'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__prochie_bilety'
]
train_df[feats_90_to_30] = train_df[feats_90_to_30]/3  # Приводим сумму за 90 дней к среднемесячным тратам за 30 дней
columns_to_sum.extend(feats_90_to_30)

# Считаем сумму трат за месяц по известным позициям
train_df['new_avg_spending_30_days'] = train_df2[columns_to_sum].sum(axis=1)
train_df['new_avg_spending_30_days'] = np.round(train_df['new_avg_spending_30_days']) # Округляем до целых чисел

test_df['new_avg_spending_30_days'] = test_df2[columns_to_sum].sum(axis=1)
test_df['new_avg_spending_30_days'] = np.round(test_df['new_avg_spending_30_days']) # Округляем до целых чисел

# Заменям на NaN все 0 в нашей новой фиче
train_df.loc[train_df['new_avg_spending_30_days'] == 0, 'new_avg_spending_30_days'] = np.nan
test_df.loc[test_df['new_avg_spending_30_days'] == 0, 'new_avg_spending_30_days'] = np.nan

### Внешние фичи

In [32]:
salary_df = pd.read_csv('data/salary.csv', sep=";", decimal=",")
salary_df = salary_df.set_index('Область')
salary_df = salary_df.drop(['2018', '2019', '2020'], axis=1)
salary_df = salary_df.rename(columns={"2022": "avg_region_salary_2022", "2021": "avg_region_salary_2021"})

salary_month_df = pd.read_csv('data/salary_month.csv', sep=";", decimal=",")
salary_month_df = salary_month_df.set_index('Область')

vpm_df = pd.read_csv('data/vpm_2024.csv', sep=";", decimal=",")
vpm_df = vpm_df.set_index('Область')
vpm_df = vpm_df.rename(columns={"2024": "vpm_2024"})

salary_rating_df = pd.read_csv('data/salary_rating.csv', sep=";", decimal=",")
salary_rating_df = salary_rating_df.set_index('Область')

rf_regions_df = pd.read_excel('data/RF_region.xlsx', decimal='.')

index_of_life_df = pd.read_excel('data/isg_2012-2023.xlsx', decimal='.', sheet_name=1)
index_of_life_df = pd.merge(index_of_life_df, rf_regions_df, on=['Город'], how='left')
index_of_life_df = index_of_life_df.groupby(by='Регион', as_index=False)[[2022, 2023]].mean()
index_of_life_df = index_of_life_df.rename(columns={'Регион': 'addrref'})
index_of_life_df.loc[len(index_of_life_df.index )] = ['Россия', 1.0, 1.0]

In [33]:
client_salary_month_df = train_df.join(salary_month_df, on='addrref')
client_salary_month_df['feature_date_month_year'] = client_salary_month_df['feature_date'].apply(lambda time: str(time.month) + '.' + str(time.year))
client_salary_month_df['avg_salary_month'] = client_salary_month_df.apply(lambda x: x[x['feature_date_month_year']], axis=1)
client_salary_month_df = client_salary_month_df[['client_id', 'avg_salary_month']]
client_salary_month_df = client_salary_month_df.set_index('client_id')

train_df = train_df.join(client_salary_month_df, on='client_id', how='left')
train_df = train_df.join(vpm_df, on='addrref', how='left')
train_df = train_df.join(salary_df, on='addrref', how='left')
train_df = train_df.join(salary_rating_df, on='addrref', how='left')
train_df = pd.merge(train_df, index_of_life_df, on=['addrref'], how='left')

test_df = test_df.join(client_salary_month_df, on='client_id', how='left')
test_df = test_df.join(vpm_df, on='addrref', how='left')
test_df = test_df.join(salary_df, on='addrref', how='left')
test_df = test_df.join(salary_rating_df, on='addrref', how='left')
test_df = pd.merge(test_df, index_of_life_df, on=['addrref'], how='left')


In [34]:
train_df.shape, test_df.shape

((205962, 263), (37183, 261))

In [35]:
# Создаём 6 новых колонок на основе комбинации помесячной даты с (ключевой ставкой, инфляцией, курсом доллара)
dates = sorted(train_df['feature_date'].unique())

key_rate = [0.08, 0.07773, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.07786, 0.10478, 0.12476]
inflation = [0.143, 0.1368, 0.1263, 0.1198, 0.1194, 0.1177, 0.1099, 0.0351, 0.0231, 0.0251, 0.0325, 0.043, 0.0515, 0.06]
usd = [60.3919, 59.8215, 61.1158, 60.853, 65.815, 68.876, 72.7828, 76.0353, 80.9955, 79.2126, 83.3193, 90.474, 95.2853, 96.6223]

# Создаём 4 новые колонки на основе комбинации помесячной даты с (денежной массы по данным центробанка)
broad_money = [85693.0, 86137.8, 86655.5, 87797.6, 94715.5, 94495.6, 97054.6, 97775.0, 97850.4, 99238.6, 100918.6, 102686.8, 105446.2, 106149.3 ] # Широкая денежная масса млрд рублей
broad_money_sa = [86072.7, 86684.4, 88206.7, 88950.2, 92377.3, 93214.9, 95150.8, 96519.8, 97722.5, 99925.5, 102253.4, 104046.6, 105877.8, 106871.4 ] # Широкая денежная масса млрд рублей сезонно скорректированная

init_cols = train_df.columns

for name, feature in zip(['key_rate', 'inflation', 'usd', 'broad_money', 'broad_money_sa'], [key_rate, inflation, usd, broad_money, broad_money_sa]):
    
    dates_dict = dict(map(lambda i,j : (i,j), dates, feature))
    train_df['new_prev_month_avg_' + name] = train_df['feature_date'].apply(lambda x: dates_dict[x])
    test_df['new_prev_month_avg_' + name] = [feature[-2]] * len(test_df)

    dates_dict = dict(map(lambda i,j : (i,j), dates, feature[1:]))
    train_df['new_this_month_avg_' + name] = train_df['feature_date'].apply(lambda x: dates_dict[x])
    test_df['new_this_month_avg_' + name] = [feature[-1]] * len(test_df)

In [37]:
features = list(train_df.drop(['client_id', 'feature_date', 'target', 'w'], axis=1).columns)

cat_features = list(train_df[features].select_dtypes('object').columns)
num_features = list(set(features) - set(cat_features))

len(features), len(cat_features), len(num_features)

(269, 11, 258)

# Сохранение датасета

In [38]:
train_df.to_parquet('data/train.parquet')
test_df.to_parquet('data/test.parquet')

d:\ml\income-prediction-alfa-campus\.venv\Lib\site-packages\pandas\io\parquet.py:189: UserWarning:

The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.

